In [ ]:
import shutil
import os
    
source_dir = 'datasets_labeled'  
file_names = os.listdir(source_dir)
file_names = [file for file in file_names if file.endswith('.xlsx')]

for ind, filename in enumerate(file_names):
    target_dir = "datasets_labeled/removed/missing_"+ os.path.splitext(filename)[0] +"/"
    os.makedirs(target_dir)
    for index, file_name in enumerate(file_names):
        if index != ind:
            shutil.copy(os.path.join(source_dir, file_name), target_dir)
 
n = 0
for ind, filename in enumerate(file_names):
    target_dir = "datasets/removed/missing_"+ os.path.splitext(filename)[0] +"/"
    os.makedirs(target_dir)
    n = n+1
        
   

In [ ]:
import os
rootdir = 'datasets_labeled/removed'

for [subdir, dirs, files], [subdir1, dirs1, files1] in zip(os.walk(rootdir), os.walk('datasets/removed')):
    print(subdir, str(subdir1))
        

In [ ]:
def get_all_pairs_from_file(directory, filename):
    term = filename.split('.')[0]
    df_rawgroups = read_excel(directory + filename, sheet_name="groups", index_col=None, header=None, engine='openpyxl')
    df_rawgroups = df_rawgroups.drop(df_rawgroups.columns[[0, 1]], axis=1)  # removing first and third unwanted column
    df_rawgroups = df_rawgroups.drop([0, 1])  # removing first and third unwanted column

    df_group_similarities = read_excel(directory + filename, sheet_name="group similarities", index_col=None, header=None, engine='openpyxl')
    df_group_similarities = df_group_similarities.drop(0, axis=0)
    df_group_similarities = df_group_similarities.drop(0, axis=1)

    df_definition = df_rawgroups[df_rawgroups.columns[range(0, df_rawgroups.shape[1]-1, 4)]]

    groups = []
    for column in df_definition.columns:
        group = []
        for definition in df_definition[column]: 
            if not pd.isna(definition): group.append(definition)
        groups.append(group)    
        pairs = []

    for grp_index1, group1 in enumerate(groups):
        for definition1 in group1:
            for grp_index2, group2 in enumerate(groups):
                for definition2 in group2:
                    if definition1 is not definition2:
                        sim = df_group_similarities.iloc[grp_index1,grp_index2]
                        if type(sim) is str: sim = sim.upper() 
                        if len(str(sim)) == 1 and (grp_index1 <= grp_index2) and not pd.isna(sim): 
                            pairs.append([term, term + str(grp_index1), term + str(grp_index2), definition1, definition2, sim])
    return pairs

def generationProcess(dataset_size, directory,loc_generated_devset,loc_generated_trainset,loc_generated_testset):
    pairs = []                  
    for filename in os.listdir(directory):
        if not filename.startswith("~") and filename.endswith(".xlsx"):
            print(os.path.join(directory, filename))
            pairs.extend(get_all_pairs_from_file(directory, filename))
        else:
            continue

    df_pairs = pd.DataFrame(pairs, columns=["term", "grp_id1", "grp_id2", "definition1", "definition2", "sim"])
    
    print(90*'=')
    print("Datenset zusammenstellen: Möglichst ausgeglichene Anzahl von similarity labels,\nBenennungen und Begriffsgruppen erstellen")
    print(90*'=' + "\n")

    df = df_pairs.copy(deep=True)
    df.insert(df.shape[1], "count_sim", [0]*df.shape[0], True) 
    df.insert(df.shape[1], "count_term", [0]*df.shape[0], True) 
    df.insert(df.shape[1], "count_grpid1", [0]*df.shape[0], True) 
    df.insert(df.shape[1], "count_grpid2", [0]*df.shape[0], True) 
    df.insert(df.shape[1], "count_def", [0]*df.shape[0], True) 

    df_dataset = pd.DataFrame(columns=df.columns)

    for i in range (0, dataset_size):
        #erstes Paar auswählen und entfernen
        if len(df) == 0:
            continue
        select = df.iloc[0]
        df_dataset = df_dataset.append(select)
        df.drop(select.name, inplace=True, errors='raise')

        #eigenschaften hochzählen
        df.loc[df.sim == select.sim, 'count_sim'] += 1
        df.loc[df.term == select.term, 'count_term'] += 1
        df.loc[df.grp_id1 == select.grp_id1, 'count_grpid1'] += 1
        df.loc[df.grp_id1 == select.grp_id2, 'count_grpid2'] += 1
        df.loc[df.grp_id2 == select.grp_id1, 'count_grpid1'] += 1
        df.loc[df.grp_id2 == select.grp_id2, 'count_grpid2'] += 1
        df.loc[df.definition1 == select.definition1, 'count_def'] += 1
        df.loc[df.definition1 == select.definition2, 'count_def'] += 1
        df.loc[df.definition2 == select.definition1, 'count_def'] += 1
        df.loc[df.definition2 == select.definition2, 'count_def'] += 1

        #sortieren, so dass Paare mit Eigenschaften, die am seltensten vertreten sind, oben stehen 
        df.sort_values(by=['count_sim','count_def','count_grpid1','count_grpid2','count_term'], inplace=True)


    #Verteilung ausgeben
    print(df_dataset.groupby(['sim']).size())
    print('')
    print(df_dataset.groupby(['term']).size())
    
    print(90*'=')
    print("Datenset speichern")
    print(90*'=' + "\n")
    from sklearn.model_selection import train_test_split

    testset_terms = ["domain", "hazard", "group", "event"]

    train_df = df_dataset[~df_dataset["term"].isin(testset_terms)]
    test_df = df_dataset[df_dataset["term"].isin(testset_terms)]
    test_df.to_csv(loc_generated_testset)


    train, dev = train_test_split(train_df, test_size=0.2)
    train.to_csv(loc_generated_trainset)
    dev.to_csv(loc_generated_devset)

    #for term in df_dataset["term"].unique():
    #    df_dataset[df_dataset["term"] == term].to_csv("datasets_by_term/"+term+".csv")
        
    #test_df
    
    #train_df[train_df["term"]=="client"]
    
    


In [ ]:
import csv
from itertools import product
from pandas import read_excel
import numpy as np
import pandas as pd
import os

# Number of pairs
dataset_size = 7000

rootdir = 'datasets_labeled/removed/'
generated_dataset_dir = 'datasets/removed/'
count = 0

for subdir1, dirs1, files1 in os.walk(generated_dataset_dir):
    if subdir1 != generated_dataset_dir and not subdir1.endswith(".ipynb_checkpoints"):
        directory = rootdir+subdir1[17:]+"/"

        print(90*'=')
        print("Generating dataset in directory: " + subdir1 + "--> "+str(count+1)+"/"+str(n))
        print(90*'=' + "\n")
    
        loc_generated_devset = subdir1 + "/dev_set.csv"
        loc_generated_trainset = subdir1 + "/train_set.csv"
        loc_generated_testset = subdir1 + "/test_set.csv"
        
        generationProcess(dataset_size, directory,loc_generated_devset,loc_generated_trainset,loc_generated_testset)
        
        count=count+1


In [ ]:
import csv
from pandas import read_csv
from itertools import product
from pandas import read_excel
import numpy as np
import pandas as pd
import os

def gen_dataframe(dataset_dir, filename, split):
    file = dataset_dir + filename
    col_list = ["sim", "definition1", "definition2"]
    dataframe = read_csv(file, usecols=col_list)
    dataframe["split"] = split
    #dataframe.loc[dataframe["sim"] == "1", "sim"] = [1.0]
    #dataframe.loc[dataframe["sim"] == "A", "sim"] = [0.8]
    #dataframe.loc[dataframe["sim"] == "B", "sim"] = [0.6]
    #dataframe.loc[dataframe["sim"] == "C", "sim"] = [0.4]
    #dataframe.loc[dataframe["sim"] == "D", "sim"] = [0.0]
    dataframe.columns = ['sentence1', 'sentence2', 'score', 'split']
    
    dataframe["score"] = [str(i).replace(",", "") for i in dataframe["score"]]
    
    #dataframe['score']=dataframe['score'].to_string()
    
   
    return dataframe

dataset_dir = 'datasets/removed/'

for subdir1, dirs1, files1 in os.walk(dataset_dir):
    if subdir1 != dataset_dir and not subdir1.endswith(".ipynb_checkpoints"):
        directory = dataset_dir+subdir1[17:]+"/"
  
        
        frames = [gen_dataframe(directory,"train_set.csv", "train"), gen_dataframe(directory,"test_set.csv", "test"), gen_dataframe(directory,"dev_set.csv", "dev")]
        combined = pd.concat(frames)
        combined["index"] = range(0, 0+len(combined))
        combined.set_index("index", inplace=True)
        combined.to_csv(directory + '/' + "gold_dataset.tsv", sep="\t", index=False)

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv

def evaluate(dataset_path, trained_model='fine-tuned_model/fine-tuning-distilroberta-base-paraphrase-v1-2021-05-10_14-17-46'):
    train_samples = []
    dev_samples = []
    test_samples = []

    with open(dataset_path, 'rt', encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            if row['score'] == "1":
                score = 1
            elif row['score'] == "A":
                score = 0.8
            elif row['score'] == "B":
                score = 0.6
            elif row['score'] == "C":
                score = 0.4
            else: score = 0
    

            inp_example = InputExample(texts=[str(row['sentence1']), str(row['sentence2'])], label=score)

            test_samples.append(inp_example)


    ##############################################################################
    #
    # Load the model and evaluate its performance on test dataset
    #
    ##############################################################################
    model = SentenceTransformer('distilroberta-base-paraphrase-v1')
    #model = SentenceTransformer(trained_model)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='test')
    test_evaluator(model, output_path=trained_model)


dataset_dir = 'datasets/removed/'

for subdir1, dirs1, files1 in os.walk(dataset_dir):
    if subdir1 != dataset_dir and not subdir1.endswith(".ipynb_checkpoints"):
        directory = dataset_dir+subdir1[17:]+"/"+"gold_dataset.tsv"
        
        print(subdir1[17:])
        evaluate(directory)
        

In [ ]:
from pandas import read_csv
import pandas

file = "plots/With_missing_terms_new_on_base_model.csv"
col_list = ["cosine_spearman"]
dataframe = read_csv(file, usecols=col_list)


x_axis = []
dataset_dir = 'datasets/removed/'
for subdir1, dirs1, files1 in os.walk(dataset_dir):
    if subdir1 != dataset_dir and not subdir1.endswith(".ipynb_checkpoints"):
        directory = dataset_dir+subdir1[17:]+"/"+"gold_dataset.tsv"  
        x_axis.append(str(subdir1[17:]))

        
df=pandas.concat([dataframe, pd.DataFrame(x_axis, columns=['on_base_model'])],axis=1)
df.to_csv("plots/plot2_values_new.csv")
fig=df.plot.bar(x='on_base_model', y='cosine_spearman',figsize=(12, 8), rot=90).get_figure()
fig.savefig("plots/plot2_new_on_base_model.jpg",bbox_inches='tight')


In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv

def evaluate(dataset_path, threshold, trained_model='fine-tuned_model/fine-tuning-distilroberta-base-paraphrase-v1-2021-05-10_14-17-46'):
    train_samples = []
    dev_samples = []
    test_samples = []

    with open(dataset_path, 'rt', encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in reader:
            if row['score'] == "1":
                score = 1
            elif row['score'] == "A":
                score = 0.8
            elif row['score'] == "B":
                score = 0.6
            elif row['score'] == "C":
                score = 0.4
            else: score = 0
                
            if score >= threshold:
                score = 1
            else: score = 0

            inp_example = InputExample(texts=[str(row['sentence1']), str(row['sentence2'])], label=score)

            test_samples.append(inp_example)


    ##############################################################################
    #
    # Load the model and evaluate its performance on test dataset
    #
    ##############################################################################
    model = SentenceTransformer('distilroberta-base-paraphrase-v1')
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='test')
    test_evaluator(model, output_path=trained_model)


dataset_dir = 'datasets/gold_dataset.tsv'

for threshold in np.arange(0.2, 0.9, 0.05):
    evaluate(dataset_dir, threshold)
        

In [ ]:
from pandas import read_csv
import pandas

file = "plots/varied_threshold_on_base_model.csv"
col_list = ["cosine_spearman"]
dataframe = read_csv(file, usecols=col_list)


x_axis = []
dataset_dir = 'datasets/removed/'
for threshold in np.arange(0.2, 0.9, 0.05): 
    x_axis.append(threshold)

        
df=pandas.concat([dataframe, pd.DataFrame(x_axis, columns=['threshold (evaluation on base model)'])],axis=1)
df.to_csv("plots/plot4_values.csv")
fig=df.plot(x='threshold (evaluation on base model)', y='cosine_spearman',figsize=(12, 8), rot=90).get_figure()
fig.savefig("plots/plot4_on_base_model.jpg",bbox_inches='tight')

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
import pandas as pd

def evaluate(dataset_path, trained_model='fine-tuned_model/fine-tuning-distilroberta-base-paraphrase-v1-2021-05-10_14-17-46'):
    train_samples = []
    dev_samples = []
    test_samples = []
    
    newdf = pd.DataFrame(columns=['sentence1','sentence2','split','score','finetuned_model_sim','base_model_sim'])
    model = SentenceTransformer(trained_model)
    model1 = SentenceTransformer('distilroberta-base-paraphrase-v1')
    with open(dataset_path, 'rt', encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)

        for row in reader:
            if row['score'] == "1":
                score = 1
            elif row['score'] == "A":
                score = 0.8
            elif row['score'] == "B":
                score = 0.6
            elif row['score'] == "C":
                score = 0.4
            else: score = 0

            #inp_example = InputExample(texts=[str(row['sentence1']), str(row['sentence2'])], label=score)

            #test_evaluator = EmbeddingSimilarityEvaluator(sentences1=[str(row['sentence1'])], sentences2=[str(row['sentence2'])], scores=[score])
            #test_evaluator(model, output_path=trained_model)
            embed=model.encode([str(row['sentence1']), str(row['sentence2'])])
            embed1=model1.encode([str(row['sentence1']), str(row['sentence2'])])
            cosine_score = util.pytorch_cos_sim(embed[0], embed[1])
            cosine_score1 = util.pytorch_cos_sim(embed1[0], embed1[1])
            df = pd.DataFrame([[row['sentence1'], row['sentence2'], row['split'], score, float(cosine_score), float(cosine_score1)]], columns=['sentence1','sentence2','split','score','finetuned_model_sim','base_model_sim'])
            newdf=pandas.concat([newdf, df], ignore_index=True)
     
    newdf.to_csv('datasets/gold_dataset1.tsv', index=False)


dataset_dir = 'datasets/gold_dataset.tsv'

evaluate(dataset_dir)
    

In [ ]:
from pandas import read_excel
import numpy as np
import pandas as pd
import os

# Number of pairs
dataset_size = 7000

rootdir = 'datasets_labeled/removed/'
generated_dataset_dir = 'datasets/removed/'

for [subdir, dirs, files], [subdir1, dirs1, files1] in zip(os.walk(rootdir), os.walk(generated_dataset_dir)):
    if subdir != rootdir and subdir1 != generated_dataset_dir and not subdir1.endswith(".ipynb_checkpoints"):
        directory = subdir+"/"
        print(subdir[0:25]+subdir1[17:])
        
        loc_generated_devset = generated_dataset_dir + "/dev_set.csv"
        loc_generated_trainset = generated_dataset_dir + "/train_set.csv"
        loc_generated_testset = generated_dataset_dir + "/test_set.csv"